In [1]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 13.0 MB/s eta 0:00:00


In [ ]:
# Importazione delle librerie necessarie
import SimpleITK as sitk
from google.colab import files

# Caricamento delle immagini .nrrd
# Questo codice apre un widget di file browser con cui puoi selezionare i file dal tuo computer.
uploaded = files.upload()


**Registrazione rigida**

In [ ]:
import SimpleITK as sitk

# Leggere le immagini usando SimpleITK
fixed_image = sitk.ReadImage('patient1.nrrd', sitk.sitkFloat32)
moving_image = sitk.ReadImage('patient2.nrrd', sitk.sitkFloat32)

# Creare un oggetto di registrazione
registration_method = sitk.ImageRegistrationMethod()

# Configurazione della metrica di somiglianza
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)

# Configurazione dell'ottimizzatore
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)

# Inizializzazione della trasformazione
initial_transform = sitk.CenteredTransformInitializer(fixed_image,
                                                      moving_image,
                                                      sitk.Euler3DTransform(),
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

registration_method.SetInitialTransform(initial_transform)

# Configurazione dell'interpolatore
registration_method.SetInterpolator(sitk.sitkLinear)

# Configurazione delle opzioni di registrazione
registration_method.SetOptimizerScalesFromPhysicalShift()

# Avvio della registrazione
final_transform = registration_method.Execute(fixed_image, moving_image)

# Applicare la trasformazione all'immagine mobile per allinearla con l'immagine fissa
moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

# Salvataggio dell'immagine risultante se necessario
sitk.WriteImage(moving_resampled, 'fixed_registration.nrrd')

**Registrazione deformabile**

In [5]:
import SimpleITK as sitk

# Leggi le immagini fisse e mobili
fixed_image = sitk.ReadImage('fixed_registration.nrrd', sitk.sitkFloat32)
moving_image = sitk.ReadImage('patient1.nrrd', sitk.sitkFloat32)

# Inizializza il metodo di registrazione
registration_method = sitk.ImageRegistrationMethod()

# Registrazione per allineamento dei centri
initial_transform = sitk.CenteredTransformInitializer(fixed_image,
                                                      moving_image,
                                                      sitk.Euler3DTransform(),
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
registration_method.SetInitialTransform(initial_transform, True)

# Primo stadio: registrazione bspline con impostazioni generali
bspline_transform = sitk.BSplineTransformInitializer(image1=fixed_image,
                                                     transformDomainMeshSize=[2, 2, 2],
                                                     order=3)
registration_method.SetInitialTransform(bspline_transform, False)

# Imposta l'ottimizzatore
registration_method.SetOptimizerAsLBFGSB(gradientConvergenceTolerance=1e-5,
                                         numberOfIterations=100,
                                         maximumNumberOfCorrections=5,
                                         maximumNumberOfFunctionEvaluations=1000,
                                         costFunctionConvergenceFactor=1e+7)

# Imposta il metodo di metrica
registration_method.SetMetricAsMeanSquares()

# Imposta i parametri di convergenza
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)

# Esegui la registrazione per il primo stadio
registration_method.Execute(fixed_image, moving_image)

# Secondo stadio: affinamento della registrazione con maggiore risoluzione della griglia
bspline_transform_refined = sitk.BSplineTransformInitializer(image1=fixed_image,
                                                             transformDomainMeshSize=[1, 1, 1],
                                                             order=3)
registration_method.SetInitialTransform(bspline_transform_refined, False)

# Esegui la registrazione per il secondo stadio
registration_method.Execute(fixed_image, moving_image)

# Salva la trasformazione risultante
sitk.WriteTransform(bspline_transform_refined, 'deformable_transform.tfm')

# Applica la trasformazione all'immagine mobile e salvala
resampled = sitk.Resample(moving_image, fixed_image, bspline_transform_refined, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
sitk.WriteImage(resampled, 'registered_patient1.nrrd')

print("Registrazione deformabile completata.")

Registrazione deformabile completata.
